In [1]:
import pandas as pd
import numpy as np

In [7]:
# Read eqtl
eqtl = pd.read_csv('../github/data/GTEx_Analysis_v7_eQTL_allTissues_slope_top.csv.gz')
eqtl = eqtl.drop(columns=['gene_id', 'variant_id'])
print(eqtl.shape)
eqtl.head()

(1349165, 48)


,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,...,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,Cells_EBV-transformed_lymphocytes
0,NaN,NaN,NaN,NaN,NaN,-0.146567,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.692326,0.712692,1.14229,0.531796,NaN,NaN,NaN,0.689146,0.593498,NaN,...,0.941748,NaN,0.826604,0.658147,0.427333,0.473418,NaN,NaN,0.794383,1.03219
3,NaN,NaN,NaN,NaN,-0.591224,-0.307221,NaN,NaN,NaN,NaN,...,-0.379419,NaN,NaN,-0.374231,NaN,-0.439161,NaN,NaN,-0.317301,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.225490,NaN


# Prepare for figure 2

In [8]:
# Get labels
y_label = 'Cells_EBV-transformed_lymphocytes'
X_label = list(i for i in eqtl.columns if i!=y_label)

In [79]:
# Get the data the eQTL for that tissue
data = eqtl[np.invert(pd.isna(eqtl[y_label]))]
# Get the eQTL that are not specific
data = data[np.invert(pd.isna(data[X_label])).sum(axis=1) != 0]

In [85]:
mean = data[X_label].mean(axis=1)
mean[:3]

2     0.767081
47   -0.908906
49   -0.578122
dtype: float64

In [86]:
howMany = np.invert(pd.isna(data[X_label])).sum(axis=1)
print(how_many[:3])

2     26
47    46
49    36
dtype: int64


In [87]:
pd.DataFrame({'RealValue':data[y_label], 'MeanImputed':mean, 'HowManyTissues':howMany}).to_csv('../github/data/data_figure2.csv', index=False)

# Prepare for figure 3

In [2]:
import os

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
readRDS = robjects.r['readRDS']

from sklearn.metrics import pairwise_distances, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor

from scipy.stats import linregress, ttest_rel

In [3]:
tissues_id = []
with open('../github/data/GTEx_v7_Annotations_TissuesId.txt') as f:
    for line in f:
        tissues_id.append(line.strip())

### Weights by PCA

In [4]:
pc = readRDS('../github/objects/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm_top_pcsX.rds')
pc = pandas2ri.ri2py(pc)
pc = pd.DataFrame(pc, columns=['PC1', 'PC2'])
pc['tissue'] = tissues_id
pc.head()

,PC1,PC2,tissue
0,-68.514239,5.674978,Adipose_Subcutaneous
1,-55.787419,0.207527,Adipose_Subcutaneous
2,-52.186166,-14.057167,Adipose_Subcutaneous
3,-48.939190,10.501181,Adipose_Subcutaneous
4,-23.998738,5.331621,Adipose_Subcutaneous


In [5]:
centroids_pc = pc.groupby('tissue')['PC1','PC2'].apply(np.mean)
centroids_pc.shape

(53, 2)

In [19]:
df_pc = pd.DataFrame(1/pairwise_distances(centroids_pc), columns=centroids_pc.index, index=centroids_pc.index).replace(np.inf, 0)

/home/lau/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


### Weights by Tsne

In [21]:
ts = readRDS('../github/objects/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm_top_tsne.rds')
ts = pandas2ri.ri2py(ts)
ts = pd.DataFrame(ts, columns=['x', 'y'])
ts['tissue'] = tissues_id
ts.head()

,x,y,tissue
0,25.209791,21.142145,Adipose_Subcutaneous
1,22.632450,15.242824,Adipose_Subcutaneous
2,19.787278,7.927666,Adipose_Subcutaneous
3,25.956711,3.907033,Adipose_Subcutaneous
4,30.342241,17.143636,Adipose_Subcutaneous


In [22]:
centroids_ts = ts.groupby('tissue')['x','y'].apply(np.mean)
centroids_ts.shape

(53, 2)

In [23]:
df_ts = pd.DataFrame(1/pairwise_distances(centroids_ts), columns=centroids_ts.index, index=centroids_ts.index).replace(np.inf, 0)

/home/lau/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


In [75]:
eqtl_s = eqtl[np.invert(pd.isna(eqtl)).sum(axis=1) != 1]
print(eqtl_s.shape)
eqtl.head()

(723420, 48)


,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,...,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,Cells_EBV-transformed_lymphocytes
0,NaN,NaN,NaN,NaN,NaN,-0.146567,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.692326,0.712692,1.14229,0.531796,NaN,NaN,NaN,0.689146,0.593498,NaN,...,0.941748,NaN,0.826604,0.658147,0.427333,0.473418,NaN,NaN,0.794383,1.03219
3,NaN,NaN,NaN,NaN,-0.591224,-0.307221,NaN,NaN,NaN,NaN,...,-0.379419,NaN,NaN,-0.374231,NaN,-0.439161,NaN,NaN,-0.317301,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.225490,NaN


In [76]:
f_e = open('../data_RMSE_figure3.tab', 'w')
f_r = open('../data_rval_figure3.tab', 'w')
print("Tissue\tn_eQTL\tRF\tmean\tpca\ttsne", file=f_e)
print("Tissue\tn_eQTL\tRF\tmean\tpca\ttsne", file=f_r)

for tissue in eqtl_s.columns:
    
    print(tissue, end='\t')
    y_label = tissue
    W = df_pc[y_label]
    U = df_ts[y_label]


    X_label = list(data.columns)
    X_label.remove(y_label)

    data = eqtl_s[np.invert(pd.isna(eqtl_s[y_label]))]
    print(data.shape)
    data = data[np.invert(pd.isna(data[X_label])).sum(axis=1) != 1]
    print(data.shape)
    
    # Split data
    X = data[X_label]
    y = data[y_label]
    X_train, X_test, y_train, y_test = train_test_split(X.fillna(0), y.fillna(0), test_size=0.3, random_state=12)

    print('Model...')
    # Model
    model = RandomForestRegressor()
    model = model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Regression
    x, y = y_test, y_pred
    slope, intercept, r_value, p_value, std_err = linregress(x,y)
    r_model = r_value
    e_model = np.sqrt(mean_squared_error(x, y))
    
    
    print('Mean...', end='\t')
    # Normal mean
    mean = X[X_label].mean(axis=1).to_list()

    # Regression
    x, y = data[y_label], mean
    slope, intercept, r_value, p_value, std_err = linregress(x,y)
    r_mean = r_value
    e_mean = np.sqrt(mean_squared_error(x, y))
    
    print('PCA...', end='\t')
    # PCA
    weighted_mean = []
    for i, r in X.iterrows():
        w = W[r.keys()[np.invert(pd.isna(r))]]
        w = w/w.sum()
        m = 0
        for t, v in r.iteritems():
            if not pd.isna(v):
                m +=  w[t]*v
        weighted_mean.append(m)
        
    # Regression
    x, y = data[y_label].values, weighted_mean
    slope, intercept, r_value, p_value, std_err = linregress(x,y)
    r_pca = r_value
    e_pca = np.sqrt(mean_squared_error(x, y))
    
    print('tSNE...')
    # tSNE
    weighted_mean = []
    for i, r in X.iterrows():
        
        u = U[r.keys()[np.invert(pd.isna(r))]]
        u = u/u.sum()
        m = 0
        for t, v in r.iteritems():
            if not pd.isna(v):
                m +=  u[t]*v
        weighted_mean.append(m)
        
    # Regression
    x, y = data[y_label].values, weighted_mean
    slope, intercept, r_value, p_value, std_err = linregress(x,y)
    r_tsne = r_value
    e_tsne = np.sqrt(mean_squared_error(x, y))

    print("{}\t{}\t{:.4f}\t{:.4f}\t{:.4f}\t{:.4f}".format(y_label, data.shape[0], e_model, e_mean, e_pca, e_tsne), file=f_e)
    print("{}\t{}\t{:.4f}\t{:.4f}\t{:.4f}\t{:.4f}".format(y_label, data.shape[0], r_model, r_mean, r_pca, r_tsne), file=f_r)

f_e.close()
f_r.close()

Adipose_Subcutaneous	(257179, 48)
(239850, 48)
Model...


/home/lau/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Mean...	PCA...	tSNE...
Adipose_Visceral_Omentum	(161362, 48)
(156976, 48)
Model...


/home/lau/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Mean...	PCA...	tSNE...
Adrenal_Gland	(76716, 48)
(72670, 48)
Model...


...

Mean...	PCA...	tSNE...
Whole_Blood	(203605, 48)
(177680, 48)
Model...


/home/lau/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Mean...	PCA...	tSNE...
Cells_EBV-transformed_lymphocytes	(48145, 48)
(46321, 48)
Model...


/home/lau/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Mean...	PCA...	tSNE...
